### Preamble

In [1]:
# Configure libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
import geohash
from sklearn.metrics import mean_squared_error

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=2, score_func=None):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds)
    gs.fit(X, y)
    print "BEST", gs.best_params_, gs.best_score_, gs.grid_scores_
    best = gs.best_estimator_
    return best

###  Reading the data

In [15]:
# Each line is of the format:
# ((time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend, geohash), number of pickups)
names = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat", "day_num", "day_cos", "day_sin", "weekend", "geohash", "pickups"]
dftaxi=pd.read_csv("./data/taxi_data.csv", header=None, names = names)
print dftaxi.shape
dftaxi = dftaxi.sample(n=10000)
# n = 50000
# dftaxi = dftaxi.head(n)
dftaxi.head()

(312330, 11)


,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,geohash,pickups
207128,11:00,0.479167,-0.991445,0.130526,Sunday,0.925595,0.892698,-0.450655,1,dr5ryx,1
199068,13:00,0.562500,-0.923880,-0.382683,Monday,0.080357,0.875223,0.483719,0,dr727s,1
68263,21:00,0.895833,0.793353,-0.608761,Monday,0.127976,0.693761,0.720205,0,dr72h7,3
173830,17:00,0.729167,-0.130526,-0.991445,Friday,0.675595,-0.450655,-0.892698,0,dr5r9j,1
203508,05:00,0.229167,0.130526,0.991445,Sunday,0.889881,0.770036,-0.638000,1,dr72wy,2


### Define training and test sets

In [16]:
itrain, itest = train_test_split(xrange(dftaxi.shape[0]), train_size=0.8)
mask=np.ones(dftaxi.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True, False,  True,  True,  True, False,  True,  True,  True,
        True])

### Final preperation for machine learning

In [17]:
# Split off the features
Xnames = ["time_cat", "time_num", "time_cos", "time_sin", "day_cat",
          "day_num", "day_cos", "day_sin", "weekend", "geohash"]
X = dftaxi[Xnames]

# Split off the target (which will be the logarithm of the number of pickups (+1))
y = np.log10(dftaxi['pickups']+1)

In [18]:
# Get the longitude and latitude from the geohash
def decodegeo(geo, which):
    if len(geo) == 6:
        geodecoded = geohash.decode(geo)
        return geodecoded[which]
    else:
        return 0
X['latitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 0))
X['longitude'] = X['geohash'].apply(lambda geo: decodegeo(geo, 1))

In [19]:
X.head()

,time_cat,time_num,time_cos,time_sin,day_cat,day_num,day_cos,day_sin,weekend,geohash,latitude,longitude
207128,11:00,0.479167,-0.991445,0.130526,Sunday,0.925595,0.892698,-0.450655,1,dr5ryx,40.778503,-73.888550
199068,13:00,0.562500,-0.923880,-0.382683,Monday,0.080357,0.875223,0.483719,0,dr727s,40.849915,-74.020386
68263,21:00,0.895833,0.793353,-0.608761,Monday,0.127976,0.693761,0.720205,0,dr72h7,40.800476,-73.987427
173830,17:00,0.729167,-0.130526,-0.991445,Friday,0.675595,-0.450655,-0.892698,0,dr5r9j,40.723572,-74.130249
203508,05:00,0.229167,0.130526,0.991445,Sunday,0.889881,0.770036,-0.638000,1,dr72wy,40.904846,-73.877563


In [20]:
# Create indicator variables for the hours and days of the week and drop the categorical values
# g = 5
X = X.join(pd.get_dummies(X['time_cat']))\
     .join(pd.get_dummies(X['day_cat']))\
     .drop(['time_cat','day_cat','geohash'], axis=1)
#     .join(pd.get_dummies(X['geohash'].str[:g]))\

In [21]:
X.head()
X.info() # http://pandas.pydata.org/pandas-docs/stable/faq.html

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 207128 to 263250
Data columns (total 40 columns):
time_num     10000 non-null float64
time_cos     10000 non-null float64
time_sin     10000 non-null float64
day_num      10000 non-null float64
day_cos      10000 non-null float64
day_sin      10000 non-null float64
weekend      10000 non-null int64
latitude     10000 non-null float64
longitude    10000 non-null float64
00:00        10000 non-null float64
01:00        10000 non-null float64
02:00        10000 non-null float64
03:00        10000 non-null float64
04:00        10000 non-null float64
05:00        10000 non-null float64
06:00        10000 non-null float64
07:00        10000 non-null float64
08:00        10000 non-null float64
09:00        10000 non-null float64
10:00        10000 non-null float64
11:00        10000 non-null float64
12:00        10000 non-null float64
13:00        10000 non-null float64
14:00        10000 non-null float64
15:00        10000 non-

### Get the training and test sets

In [22]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
Xtrain.head()

,time_num,time_cos,time_sin,day_num,day_cos,day_sin,weekend,latitude,longitude,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00,18:00,19:00,20:00,21:00,22:00,23:00,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
207128,0.479167,-0.991445,0.130526,0.925595,0.892698,-0.450655,1,40.778503,-73.888550,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
68263,0.895833,0.793353,-0.608761,0.127976,0.693761,0.720205,0,40.800476,-73.987427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
173830,0.729167,-0.130526,-0.991445,0.675595,-0.450655,-0.892698,0,40.723572,-74.130249,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
203508,0.229167,0.130526,0.991445,0.889881,0.770036,-0.638000,1,40.904846,-73.877563,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
85625,0.354167,-0.608761,0.793353,0.336310,-0.516106,0.856525,0,40.652161,-73.778687,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### k-Nearest Neighbors Regression

In [23]:
# Create a k-Nearest Neighbors Regression estimator
knn_estimator = KNeighborsRegressor()

#### Normalize Data

In [24]:
Xtrain_mean = Xtrain.mean()
Xtrain_std_dev = Xtrain.std()
Xtrain_normalized = (Xtrain - Xtrain_mean)/Xtrain_std_dev
Xtest_normalized = (Xtest - Xtrain_mean)/Xtrain_std_dev


In [26]:
%%time
# Define a grid of parameters over which to optimize the knn regressor
# We will figure out which number of neighbors is optimal
#knn_parameters = {"n_neighbors": [1,2,5,10,20,50,100]}
knn_parameters = {"n_neighbors": [1,2,5]}
knn_best = cv_optimize(knn_estimator, knn_parameters, Xtrain_normalized, ytrain, score_func='mean_squared_error')

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python2.7/dist-packages/sklearn/m

BEST {'n_neighbors': 2} -0.962246194237 [mean: -1.10824, std: 0.01628, params: {'n_neighbors': 1}, mean: -0.96225, std: 0.01445, params: {'n_neighbors': 2}, mean: -1.00703, std: 0.02054, params: {'n_neighbors': 5}]
CPU times: user 3.21 s, sys: 8 ms, total: 3.22 s
Wall time: 3.24 s


### k-Nearest Neighbors Regression

In [27]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
knn_reg=knn_best.fit(Xtrain_normalized, ytrain)
knn_training_accuracy = knn_reg.score(Xtrain_normalized, ytrain)
knn_test_accuracy = knn_reg.score(Xtest_normalized, ytest)
print "############# based on standard predict ################"
print "R^2 on training data: %0.4f" % (knn_training_accuracy)
print "R^2 on test data:     %0.4f" % (knn_test_accuracy)

############# based on standard predict ################
R^2 on training data: 0.8108
R^2 on test data:     0.4075


In [28]:
# Show some of the predictions vs. the real number of pickups
# predictions vs. real number of pickups
np.round(np.power(10,np.column_stack((knn_reg.predict(Xtest_normalized),ytest))) - 1,decimals=0).astype(int)

array([[ 1,  1],
       [ 1,  1],
       [ 4, 15],
       ...,
       [70, 23],
       [ 1,  1],
       [ 7,  3]])

In [29]:
# Calculate the Root Mean Squared Error
np.sqrt(mean_squared_error(knn_reg.predict(Xtest_normalized),ytest))

0.8104902194598163